# Implementación de algoritmos de aproximación offline para resolver el problema "bin packing" en una dimensión
## Introducción
// TO-DO

Los algoritmos offline para este problema que se presentan a continuación:
## Algoritmo "First-fit-decreasing" (**FFD**)
La idea de este algoritmo es acomodar cada artículo de acuerdo a su tamaño; primero los artículos más grandes y cada vez que un artículo sea muy grande para la capacidad restante de los contenedores utilizados, se utilizará un nuevo contenedor.

In [2]:
# Clase Instancia
class Instancia:

    def __init__(self, name, capacidad, tamanio, mejor_sol):
        self.name = name
        self.items = []
        self.cap_bin = int(capacidad)
        self.tam_instancia = int(tamanio)
        self.mejor_solucion = int(mejor_sol)

    def borrar_item(self, i):
        self.items.delete(i)

# Clase Contenedor
class Contenedor:

    def __init__(self, capacidad):
        self.items = []
        self.capacidad = capacidad
        self.conteo = capacidad

    def conteo_func(self):
        self.conteo = self.capacidad - sum(self.items)

    def ingresa_item(self, item):
        self.items.append(item)
        self.conteo_func()

    def get_conteo(self):
        return self.conteo


Se crearán dos listas una para los contenedores utilizados y otra para los artículos; para este ejercicio se utilizarán instancias del sitio: http://people.brunel.ac.uk/~mastjjb/jeb/orlib/binpackinfo.html contenidas en el directorio de este instructivo con el nombre `binpack1.txt`
 ### Instancia
 El formato del archivo es el siguiente, nombrado por renglón:
 1. Número de instancias en el archivo (**P**)
 1. Para cada instancia **P**
 >* Identificador del problema
 >* Capacidad del contenedor, número de artículos (**n**), mejor solución encontrada por número de contenedores
 >* Para cada artículo, su tamaño

In [3]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
def lee_archivo_instancias(archivo_nombre):
    archivo = open(archivo_nombre, 'r')
    num_instancias = int(archivo.readline())
    # Crea instancia de objeto "Instancia" para retorno
    nombre_instancia = archivo.readline()
    lista_temp = (archivo.readline()).split()
    instancia_1 = Instancia(nombre_instancia, lista_temp[0], lista_temp[1], lista_temp[2])

    for i in range(int(lista_temp[1])):
        instancia_1.items.append(int(archivo.readline()))

    archivo.close()

    # regresa una lista, capacidad del bin y la mejor solución (conocida)
    return instancia_1


instancia_1 = lee_archivo_instancias("/content/drive/MyDrive/ColabNotebooks/PT1/binpack1.txt")

Después de leer la instancia se procede a ordenar los artículos de forma decreciente para empezar a asignarlos.

In [5]:
# Se crea una lista de contenedores vacía
bins = []

#Se agrega un contenedor al bins list para empezar
container_temp = Contenedor(instancia_1.cap_bin)
bins.append(container_temp)

# se ordenan los articulos de la instancia de forma decreciente
instancia_1.items.sort(reverse=True)


# first fit decreasing
def first_fit_decreasing(bins_list, instancia):
    # Para cada elemento de la lista de items
    for i in range(len(instancia.items)):
        # Para cada contenedor
        for j in range(len(bins_list)):
            if instancia.items[i] <= bins_list[j].get_conteo():
                # Entonces se ingresa en ese bin
                bins_list[j].ingresa_item(instancia.items[i])
                flag = True
                break
            else:
                flag = False
        # pregunta si es el ultimo bin, si ya no hay bins se crea nuevo
        if flag == False:
            # crea bin nuevo
            bins_list.append(Contenedor(instancia.cap_bin))
            # ingresa en ese bin
            bins_list[len(bins_list)-1].ingresa_item(instancia.items[i])



first_fit_decreasing(bins, instancia_1)

Crearemos una función para mostrar el contenido de los contenedores y su información

In [6]:
def imprime_solucion(lista_bins):
    print("Contenedores obtenidos: ", len(lista_bins),"\n")
    """
    for i in range(len(lista_bins)):
        print("Contenedor", (i+1),"-> cap = ", lista_bins[i].conteo)
        print("Artículos : ", lista_bins[i].items)
    """

imprime_solucion(bins)

Contenedores obtenidos:  49 



---
## Algoritmo "Next-fit-decreasing" (**NFD**)
Este algoritmo funciona de la siguiente manera:
* Se ordenan los artículos desde el más grande al mas pequeño
* Se inicia un contenedor vacio
* Para cada artículo se verifica si cabe en ese contenedor
>* Si el artículo cabe en el contenedor abierto, entonces se ingresa en él
>* De lo contrario se abre un nuevo contenedor para agrega artículos y se cierra el anterior

Vamos a trabajar con la misma instancia `instancia_1`

In [7]:
# Se crea una lista de contenedores vacía
bins = []

#Se agrega un contenedor al bins list para empezar
container_temp = Contenedor(instancia_1.cap_bin)
bins.append(container_temp)

# Next fit decreasing
def next_fit_decreasing(bins_list, instancia):
    # Para cada elemento de la lista de items
    for i in range(len(instancia.items)):
        # si cabe en el contenedor abierto
        if instancia.items[i] <= bins_list[len(bins_list)-1].get_conteo():
            # Entonces se ingresa en ese bin
            bins_list[len(bins_list)-1].ingresa_item(instancia.items[i])
        # si no cabe
        else:
            # crea bin nuevo
            bins_list.append(Contenedor(instancia.cap_bin))
            # ingresa en ese bin
            bins_list[len(bins_list)-1].ingresa_item(instancia.items[i])



next_fit_decreasing(bins, instancia_1)
imprime_solucion(bins)

Contenedores obtenidos:  67 



---
## Algoritmo "Modified-first-fit-deacreasing" (**MFFD**)
Este algoritmo sigue el mismo procedimiento de agrupamiento que **FFD** pero divide el conjunto de artículos a empacar, de acuerdo a su valor, en distintos _tipos_; desde el tipo $A$ hasta el tipo $G$, en este caso. Primero ordenaremos la lista de artículos de forma decreciente, donde $L={x_1,x_2,\cdots,x_n}$ es la lista de artículos $x_i$ y $s(x_1)\geq s(x_2)\geq\cdots\geq s(x_n)$ indica el orden decreciente.

$x$                           | $tipo$   
------------------------------|-----------
$A={x:s(x) \in (1/2,1]}$      | A - items
$B={x:s(x) \in (1/3,1/2]}$    | B - items
$C={x:s(x) \in (1/4,1/3]}$    | C - items
$D={x:s(x) \in (1/5,1/4]}$    | D - items
$E={x:s(x) \in (1/6,1/5]}$    | E - items
$F={x:s(x) \in (11/71,1/6]}$  | F - items
$G={x:s(x) \in (0,11/71]}$    | G - items

### El algoritmo

Después de clasficar los artículos (_items_) en sus respectivos conjuntos (_tipos_) se aplica la rutina del **MFFD**:

1. Asignar los artículos del tipo $A$ en $|A|$ contenedores $X$, donde el _nivel_ (i.e., el tamaño total de los artículos de ese contenedor) de $X_1$ indique el mayor tamaño y $X_{|A|}$ el menor nivel. Estos serán los $A$-_contenedores_.
1. A través de los $A$-contenedores, para cada $X_i$, desde $X_1$ hasta $X_{|A|}$ hacer lo siguiente: si alguno de los artículos no empacados de tipo $B$ encaja en $X_i$ empaque el más grande en $X_i$. (Note que puede haber espacio para a lo sumo uno)
1. A través de los $A$-contenedores, para cada $X_i$, desde $X_{|A|}$ hasta $X_1$ (de derecha a izquierda) hacer lo siguiente:
>* Si $X_i$ contiene un artículo de tipo $B$, no hacer nada
>* Si los dos artículos no empacados más pequeños de $C\cup D\cup E$ no encajan juntos o si solamente queda un artículo de este tipo, no hacer nada.
>* De otra manera, empacar el artículo más pequeño no empacado de $C\cup D\cup E$ junto con el artículo, no empacado, más grande posible (que encaje) en $X_i$.
1. Proceder, una ultima vez, a través de los $A$-contenedores, para cada $X_i$, desde $X_1$ hasta $X_{|A|}$ hacer lo siguiente: si algún artículo no empacado encaja en $X_i$ empaque el artículo más grande que encaje y repetir hasta que no encaje ningun artículo no empacado.
1. Use **FFD** para empacar los artículos restantes no empacados empezando por $X_{|A|+1}$


In [ ]:
# Se vacia la lista de contenedores
bins = []


def subrutina_conjuntos(instancia, a, b): # (1/6,1/3]
    # define los dos conjuntos más pequeños de un grupo de (A , B]
    # y devuelve el valor de la suma de ambos
    penultimo = 0
    ultimo = instancia.cap_bin
    conteo = 0
    for i in range(len(instancia.items)):
        if (instancia.items[i] > (a * instancia.cap_bin)) and \
            (instancia.items[i] <= (b * instancia.cap_bin))
            if(instancia.items[i] < ultimo)
                penultimo = ultimo
                ultimo = instancia.items[i]
                conteo = conteo + 1

    if conteo == 1:
        return False
    else:
        return (ultimo + penultimo)



# modified first fit decreasing
def modified-first_fit_decreasing(bins_list, instancia):
    """ 1
    """
    # mientras el item sea A-item:
    while instancia.items[0] > (instancia.cap_bin/2):
        # mientras el item sea A-item:
        # Crear bin, agregar el A-item al A-bin, eliminar A-item[i] de items
        bins_list.append(Contenedor(instancia.cap_bin))
        bins_list[len(bins_list)-1].ingresar_item(instancia.item[0])
        instancia.borrar_item(0)
    """ 2
    """
    # A través de los A-bins de 0 a |A-item|
    for i in range(len(bins_list)):
        # Verifica los B-items no empacados, empaca el más grande que encaje
        # en A-bins_i
        for j in range(len(instancia.items)):
            if (instancia.item[j] > (instancia.cap_bin/3)) and \
             (instancia.item[j] <= (instancia.cap_bin/2)):
                # Agrega el B-item si encaja
                if instancia.item[j] <= bin_list[i].get_conteo():
                    bin_list[i].ingresar_item(instancia.item[j])
                    # elimina de L-items
                    instancia.borrar_item(j)
                    # Rompe el ciclo de los B-items y vuelve al de mayor tamaño
                    break
    """ 3
    """
    for i in range(len(bins_list), -1, -1, -1):
            # Verifica si hay B-items en X_i, verifica cada item
        for j in range(len(bins_list[i].items)):
            if (bins_list[i].items[j] > (instancia.cap_bin/3)) and \
             (bins_list[i].items[j] <= (instancia.cap_bin/2)):
                break
            # Revisa si caben los dos elementos mas pequeños de C U E U D
            elif (bins_list[i].get_conteo() < subrutina_conjuntos(instancia, (1/6), (1/3)) \
                    or subrutina_conjuntos(instancia, (1/6), (1/3)) == False):
                break
            # En otro caso:
            else:
                # Empacar el artículo más grande y el más chico no empacados
                for i in range(len(instancia.items)):







